In [1]:
import random
import copy
import numpy as np
import theano as T
import theano.tensor as tt
import pymc3 as pm
import pandas as pd
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%%R

library(ggplot2)
library(reshape)
library(grid)
library(dplyr)
library(gridExtra)
library(lme4)


paper_theme_2 <- theme_light() + theme(axis.title.x = element_text(size=18),
  axis.text.x=element_text(colour="black", 
                           size = 14), 
  axis.title.y = element_text(size = 18, vjust = 1),
  axis.text.y  = element_text(size = 14, colour="black"),
 title =element_text(size=16, face='bold'))



data_resp_loc <- read.csv("data/discrimination_dot_gaze.csv", sep="\t")
data_tracker <- read.csv("data/discrimination_tracker_data.csv", sep="\t")
data_tracker$trial_id <- floor((data_tracker$trial_id - 1)/2)
data_resp_loc$trial_id <- data_resp_loc$trial_id - 1


data_resp_loc <- data_resp_loc %>%
                filter(belowX >= 0) #%>%
                #mutate(cond=paste(round(Time1),round(Time2),sep="_"))

trials <- (unique(data_resp_loc$trial_id))
data_tracker <- data_tracker %>%
                group_by(trial_id) %>%
                filter(trial_id %in% trials)

data_resp_loc$id <- seq.int(1, nrow(data_resp_loc))
data_resp_loc$Score <- as.numeric(data_resp_loc$Score) - 1

#data_resp_loc$cond <- plyr::mapvalues(data_resp_loc$cond, 
                  #                from = c("0_0", "0_1", "1_0", "1_1"),
                        #          to = c("Short_Short", "Short_Long", "Long_Short", "Long_Long"))

data_resp_loc_old <- cbind(data_resp_loc)


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Need help getting started? Try the cookbook for R:
http://www.cookbook-r.com/Graphs/

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:reshape’:

    rename


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/pyth

In [ ]:
%%R -o data_resp_avs

data_resp_loc <- cbind(data_resp_loc_old)
        
data_resp_avs <- data_resp_loc %>%
                  filter((gazeDist > 0) & (Trial_End >= 1523639984)) %>%
                  mutate(Dots_Counted = Dots_Counted - 1) %>%
                  group_by(trial_id, which_array) %>%
                  mutate(arr1 = (which_array==1)*1) %>%
                  mutate(arr2 = (which_array==2)*1) %>%
                  mutate(belowX1 = mean(belowX) * arr1) %>%
                  mutate(belowX2 = mean(belowX) * arr2)%>%
                 # mutate(within_x1 = sum(arr1/(1+exp(-0.1*belowX)))) %>%
                  #mutate(within_x2 = sum(arr2/(1+exp(-0.1*belowX)))) %>%

                  mutate(within_x1 =  arr1 * sum( (2/(1 + exp(-2 -0.5*belowX)) - 1))) %>%
                  mutate(within_x2 =  arr2 * sum((2/(1 + exp(-2 -0.5*belowX)) - 1))) %>%
                  #mutate(within_x1_tot = sum(within_x1)) %>%
                  #mutate(within_x2_tot = sum(within_x2)) %>%

                  mutate(pctArea1 = mean(pctArea) * arr1) %>%
                  mutate(pctArea2 = mean(pctArea) * arr2)%>%
                  mutate(totArea1 = mean(totArea) * arr1) %>%
                  mutate(totArea2 = mean(totArea) * arr2)%>%
                  top_n(n=1, wt=id)%>%
                  group_by(trial_id) %>%
                  mutate(belowX1 = max(belowX1)) %>%
                  mutate(belowX2 = max(belowX2)) %>%
                  mutate(within_x1 = max(within_x1)) %>%
                  mutate(within_x2=max(within_x2)) %>%
                  #mutate(within_x1_tot = max(within_x1_tot)) %>%
                  #mutate(within_x2_tot = max(within_x2_tot)) %>%

                  mutate(pctArea1 = max(pctArea1)) %>%
                  mutate(pctArea2 = max(pctArea2)) %>%
                  mutate(totArea1 = max(totArea1)) %>%
                  mutate(totArea2=max(totArea2)) %>%
                  top_n(n=1, wt=id) %>%
                  mutate(within_x_diff = (within_x1 - within_x2)) %>%
                 mutate(within_x_density_diff = within_x1/pctArea1 - within_x2 / pctArea2)
 
p.1 <- ggplot(data=data_resp_avs, aes(x=within_x_diff, y=Dots_Counted)) +
        binomial_smooth()

p.2 <- ggplot(data=data_resp_avs, aes(x=within_x_density_diff, y=Dots_Counted)) +
        binomial_smooth()

r.1 <- glm(data=data_resp_avs, Dots_Counted ~ within_x_diff, family=binomial(link="probit"))
r.2 <- glm(data=data_resp_avs, Dots_Counted ~ within_x_density_diff, family=binomial(link="probit"))
r.3 <- glm(data=data_resp_avs, Dots_Counted ~ within_x_diff * within_x_density_diff, family=binomial(link="probit"))

summary(r.3)
#summary(lm(data_resp_avs$within_x2_tot ~ data_resp_avs$Dots_Shown2))
#print(summary(lm(data_resp_avs$pctArea2 ~ data_resp_avs$Time2)))


In [ ]:
data_resp = pandas2ri.ri2py(data_resp_avs)

In [ ]:

def remove_n(ls,n):
    ret = []
    for l in ls:
        l_rem = l[:n]
        ret.append(l_rem)
    return ret


subj_group = data_resp.groupby("pid")
dots_shown1,dots_shown2,dots_counted,pct_area1,pct_area2,seen1,seen2=[],[],[],[],[],[],[]
for i,subj in subj_group:
    dots_counted.append(list(subj["Dots_Counted"]))
    dots_shown1.append(list(subj["Dots_Shown1"]))
    pct_area1.append(list(subj["pctArea1"]))
    seen1.append(list(subj["within_x1"]))
    #seen1.append(list(subj["within_x1_tot"]))
    dots_shown2.append(list(subj["Dots_Shown2"]))
    pct_area2.append(list(subj["pctArea2"]))
    #seen2.append(list(subj["within_x2_tot"]))
    seen2.append(list(subj["within_x2"]))

ns = [len(n) for n in seen1 + seen2]
min_n = min(ns)

dots_counted = remove_n(dots_counted, min_n)
dots_shown1 = remove_n(dots_shown1, min_n)
pct_area1 = remove_n(pct_area1, min_n)
seen1 = remove_n(seen1, min_n)
dots_shown2 = remove_n(dots_shown2, min_n)
pct_area2 = remove_n(pct_area2, min_n)
seen2 = remove_n(seen2, min_n)


dots_counted=np.array(dots_counted)
dots_shown1=np.array(dots_shown1)
pct_area1=np.array(pct_area1)
seen1=np.array(seen1)
unseen1 = dots_shown1 - seen1
dots_shown2=np.array(dots_shown2)
pct_area2=np.array(pct_area2)
seen2=np.array(seen2)
unseen2 = dots_shown2 - seen2
dims = (len(seen1),len(seen1[0]))

with pm.Model() as mod:
    #GROUP-LEVEL PARAMETERS
    
    #MEAN SLOPE
    b_seen_grp = pm.Normal("b_seen_grp", mu=0,sd=10,testval=1)
    b_unseen_grp = pm.Normal("b_unseen_grp", mu=0,sd=10,testval=1)
    sd_seen_grp = pm.HalfNormal("sd_seen_grp",sd=10,testval=5)
    sd_unseen_grp = pm.HalfNormal("sd_unseen_grp",sd=10,testval=5)
    
    ##NOISE SD
    #sd_noise_grp = pm.HalfNormal("b_noise_grp", sd=10,testval=5)

    ##ALPHA HYPERPARAMS
    #theta_seen_grp = pm.Beta("theta_seen_grp", 0.25,0.25)
    #theta_unseen_grp = pm.Beta("theta_unseen_grp", 0.25,0.25)
    #lambda_seen_grp = pm.Exponential("lambda_seen_grp",1)
    #lambda_unseen_grp = pm.Exponential("lambda_unseen_grp",1)

    #SUBJECT-LEVEL PARAMETERS
    
    ##MEAN SLOPE
    b_seen_subj = pm.Normal("b_seen_subj",mu=b_seen_grp,sd=sd_seen_grp, shape=(len(seen1),1))
    b_unseen_subj = pm.Normal("b_unseen_subj",mu=b_unseen_grp,sd=sd_unseen_grp, shape=(len(unseen1),1))
    #b_seen_subj, b_unseen_subj = 1, 0
    
    ##ALPHA (DENSITY)
    #alpha_seen_subj = pm.Beta("alpha_seen_subj",
                            #     theta_seen_grp * lambda_seen_grp, 
                                 # (1-theta_seen_grp) * lambda_seen_grp, shape=(len(seen1),1))
   # alpha_unseen_subj = pm.Beta("alpha_unseen_subj",
                    #            theta_unseen_grp * lambda_unseen_grp, 
                            #      (1-theta_unseen_grp) * lambda_unseen_grp, shape=(len(unseen1),1))
    alpha_seen_subj, alpha_unseen_subj = 0,0
    
    ##NOISE
    #b_noise_subj = pm.HalfNormal("b_noise_subj", sd=sd_noise_grp, shape=(len(unseen1),1))
    b_noise_subj = 1
    
    guess_mean1 = (b_seen_subj * seen1 * (1./pct_area1) ** alpha_seen_subj + 
                  b_unseen_subj * unseen1 * (1./(1-pct_area1)) ** alpha_unseen_subj)
    guess_mean2 = (b_seen_subj * seen2 * (1./pct_area2) ** alpha_seen_subj + 
                  b_unseen_subj * unseen2 * (1./(1-pct_area2)) ** alpha_unseen_subj)
    
    
    
    ps = (guess_mean2 - guess_mean1) /  (b_noise_subj * ((guess_mean2**2 + guess_mean1**2)**0.5))

    ps = 0.5 * (1. + tt.erf(ps/(2.**0.5)))
    

    likelihood = pm.Bernoulli('likelihood', ps,  observed=dots_counted, shape=dims)

    trace = pm.sample(1000, tune=200, cores=2)


summary = pm.summary(trace)
print(summary)
    

In [ ]:
print(summary)